In [1]:
import pandas as pd
import joblib

# Load scored test data
test_df = pd.read_csv(r"E:\Projects Data Scientist\credit-risk-project\data\processed\test_scored.csv")

y_test = test_df["actual_default"]
y_test_pd = test_df["pd"]

In [2]:
# Define a Simple Business Assumption
AVG_LIMIT = 100_000
LGD = 0.6

In [3]:
# Baseline (No Model) Scenario
baseline_default_rate = test_df["actual_default"].mean()

baseline_loss = (
    baseline_default_rate *
    AVG_LIMIT *
    LGD *
    len(test_df)
)

print(f"Baseline Default Rate: {baseline_default_rate:.2%}")
print(f"Baseline Expected Loss: ₹{baseline_loss:,.0f}")

Baseline Default Rate: 6.68%
Baseline Expected Loss: ₹180,480,000


In [5]:
# Model-Based Scenario
def assign_risk_band(pd):
    if pd <= 0.05:
        return "Low"
    elif pd <= 0.15:
        return "Medium"
    else:
        return "High"

test_df["risk_band"] = test_df["pd"].apply(assign_risk_band)

approved_df = test_df[
    test_df["risk_band"].isin(["Low", "Medium"])
]

model_default_rate = approved_df["actual_default"].mean()

model_loss = (
    model_default_rate *
    AVG_LIMIT *
    LGD *
    len(approved_df)
)

print(f"Model Approval Rate: {len(approved_df)/len(test_df):.2%}")
print(f"Model Default Rate: {model_default_rate:.2%}")
print(f"Model Expected Loss: ₹{model_loss:,.0f}")

Model Approval Rate: 12.80%
Model Default Rate: 0.73%
Model Expected Loss: ₹2,520,000


In [6]:
# Loss Reduction (This Is the Money Slide)
loss_reduction = baseline_loss - model_loss

print(f"Estimated Loss Reduction: ₹{loss_reduction:,.0f}")

Estimated Loss Reduction: ₹177,960,000


In [7]:
# Summary Table
summary = pd.DataFrame({
    "Scenario": ["No Model", "With Risk Model"],
    "Approval Rate": [
        "100%",
        f"{len(approved_df)/len(test_df):.2%}"
    ],
    "Expected Loss (₹)": [
        round(baseline_loss),
        round(model_loss)
    ]
})

summary

,Scenario,Approval Rate,Expected Loss (₹)
0,No Model,100%,180480000
1,With Risk Model,12.80%,2520000
